In [45]:
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
import pandas as pd
import math

def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:80]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/80)
    
    df = makePageDF(category_id,1)
    
    for j in range(1,10):
        if total_page >1:
            for i in range(2,total_page+1):
                new_df = makePageDF(category_id,i)
                df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [43]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [44]:
df

,ID,상품명,상품 카테고리 대분류,상품 카테고리 중분류,상품 카테고리 소분류,제조사,브랜드,특징,샴푸,천연샴푸,...,보색샴푸,쿨샴푸,주니어겸용,쉐도우형,무스형,스틱형,염색 모발용,"시트,티슈형",브러쉬형,파우더형
0,28269721554,검은콩 딥 스케일링 나이트 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 천연샴푸, 비듬샴푸, 두피샴푸, 한방샴푸, 퍼퓸샴푸, 펌프형, 저자극, 촉...",True,True,...,False,False,False,False,False,False,False,False,False,False
1,28269744554,검은콩 보습 진정 모닝 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 모든두피용, 모든 모발용, 일반샴푸, 펌프형, 머릿결개선, 세정력, 풍부한...",True,False,...,False,False,False,False,False,False,False,False,False,False
2,29502248618,어글리솝 더 님 민감성용 97g,화장품/미용,헤어케어,샴푸,프롬더랜드,프롬더랜드,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급),...",True,False,...,False,False,False,False,False,False,False,False,False,False
3,20991784508,녹차실감 산뜻한 타입 지성 모발용 샴푸 500g,화장품/미용,헤어케어,샴푸,아모스,아모스,"[샴푸, 지성, 두피샴푸, 펌프형, 세정력, 상쾌함, 저자극, 촉촉함(수분공급), ...",True,False,...,False,False,False,False,False,False,False,False,False,False
4,29124648587,프로체인지 블랙샴푸 300g,화장품/미용,헤어케어,샴푸,더마밀,모다모다,"[샴푸, 탈모샴푸, 펌프형, 저자극, 계면활성제 프리, 파라벤 프리, 실리콘 프리,...",True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21742,31421590618,지케이인터내셔널 클리닉 트리트먼트 샴푸 아르간 750ml,화장품/미용,헤어케어,샴푸,,메이아일랜드,"[샴푸, 일반샴푸, 펌프형, 머릿결개선, 세정력, 풍부한 거품, 모발영양]",True,False,...,False,False,False,False,False,False,False,False,False,False
21760,7814549375,데미지 케어 샴푸 680ml,화장품/미용,헤어케어,샴푸,아모레퍼시픽,미쟝센,"[샴푸, 리퀴드형, 촉촉함(수분공급)]",True,False,...,False,False,False,False,False,False,False,False,False,False
21788,37153208620,플래티넘 앤 다이아몬드 럭셔리어스 샴푸 250ml,화장품/미용,헤어케어,샴푸,미리암퀘베도,미리암퀘베도,"[샴푸, 일반샴푸, 리퀴드형, 촉촉함(수분공급), 모발영양, 모발윤기]",True,False,...,False,False,False,False,False,False,False,False,False,False
22255,35316621618,나드 두피 딥클렌징 샴푸 900ml,화장품/미용,헤어케어,샴푸,서울화장품,부케가르니,"[샴푸, 일반샴푸, 펌프형, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급), 각...",True,False,...,False,False,False,False,False,False,False,False,False,False


In [46]:
merged_df

,ID,상품명,특징,attribute_id
0,28269721554,검은콩 딥 스케일링 나이트 샴푸 로즈마리향 400ml,샴푸,0
1,28269744554,검은콩 보습 진정 모닝 샴푸 로즈마리향 400ml,샴푸,0
2,29502248618,어글리솝 더 님 민감성용 97g,샴푸,0
3,20991784508,녹차실감 산뜻한 타입 지성 모발용 샴푸 500g,샴푸,0
4,29124648587,프로체인지 블랙샴푸 300g,샴푸,0
...,...,...,...,...
46395,40330480626,세린 스칼프 오일 컨트롤 샴푸 파우더 45g,파우더형,85
46396,32437168621,노 드라우트 드라이 샴푸 115g,파우더형,85
46397,35359117623,스템루텐스 퍼퓸 샴푸 스프링 오브 프리지아 500ml,파우더형,85
46398,39512968626,탭 시크릿 7g,파우더형,85


In [47]:
attributes

,attribute_id,특징
0,0,샴푸
1,1,천연샴푸
2,2,비듬샴푸
3,3,두피샴푸
4,4,한방샴푸
...,...,...
81,81,스틱형
82,82,염색 모발용
83,83,"시트,티슈형"
84,84,브러쉬형


In [48]:
def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:80]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/80)
    
    df = makePageDF(category_id,1)
    
    for j in range(1,5):
        if total_page >1:
            for i in range(2,total_page+1):
                new_df = makePageDF(category_id,i)
                df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [49]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [50]:
df

,ID,상품명,상품 카테고리 대분류,상품 카테고리 중분류,상품 카테고리 소분류,제조사,브랜드,특징,샴푸,천연샴푸,...,보색샴푸,쿨샴푸,주니어겸용,쉐도우형,무스형,스틱형,염색 모발용,"시트,티슈형",브러쉬형,파우더형
0,28269721554,검은콩 딥 스케일링 나이트 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 천연샴푸, 비듬샴푸, 두피샴푸, 한방샴푸, 퍼퓸샴푸, 펌프형, 저자극, 촉...",True,True,...,False,False,False,False,False,False,False,False,False,False
1,28269744554,검은콩 보습 진정 모닝 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 모든두피용, 모든 모발용, 일반샴푸, 펌프형, 머릿결개선, 세정력, 풍부한...",True,False,...,False,False,False,False,False,False,False,False,False,False
2,29502248618,어글리솝 더 님 민감성용 97g,화장품/미용,헤어케어,샴푸,프롬더랜드,프롬더랜드,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급),...",True,False,...,False,False,False,False,False,False,False,False,False,False
3,20991784508,녹차실감 산뜻한 타입 지성 모발용 샴푸 500g,화장품/미용,헤어케어,샴푸,아모스,아모스,"[샴푸, 지성, 두피샴푸, 펌프형, 세정력, 상쾌함, 저자극, 촉촉함(수분공급), ...",True,False,...,False,False,False,False,False,False,False,False,False,False
4,29124648587,프로체인지 블랙샴푸 300g,화장품/미용,헤어케어,샴푸,더마밀,모다모다,"[샴푸, 탈모샴푸, 펌프형, 저자극, 계면활성제 프리, 파라벤 프리, 실리콘 프리,...",True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21576,32235714619,모링가 체리블라썸 샴푸 1000g,화장품/미용,헤어케어,샴푸,바이원코스메틱,바론,"[샴푸, 일반샴푸, 펌프형, 머릿결개선, 상쾌함, 저자극, 각질케어, 유분제거, 비...",True,False,...,False,False,False,False,False,False,False,False,False,False
21687,34288965618,라디아 딥클렌징 샴푸 205ml,화장품/미용,헤어케어,샴푸,DS래보래토리즈,DS래보래토리즈,"[샴푸, 촉촉함(수분공급)]",True,False,...,False,False,False,False,False,False,False,False,False,False
21717,13889601388,하나로케어 스프레이형 샴푸 250ml,화장품/미용,헤어케어,샴푸,,,[샴푸],True,False,...,False,False,False,False,False,False,False,False,False,False
21834,29109296588,카이라스 컬러 프로텍션 염색모용 샴푸 1000ml,화장품/미용,헤어케어,샴푸,머스터앤딕슨,딕슨,"[샴푸, 일반샴푸, 펌프형, 머릿결개선, 세정력, 파라벤 프리, 실리콘 프리, 약산...",True,False,...,False,False,False,False,False,False,False,False,False,False


In [51]:
def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '40',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:40]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '40',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/40)
    
    df = makePageDF(category_id,1)
    
    if total_page >1:
        for i in range(2,total_page+1):
            new_df = makePageDF(category_id,i)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [52]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [53]:
df

,ID,상품명,상품 카테고리 대분류,상품 카테고리 중분류,상품 카테고리 소분류,제조사,브랜드,특징,샴푸,천연샴푸,...,보색샴푸,쿨샴푸,주니어겸용,쉐도우형,무스형,스틱형,염색 모발용,"시트,티슈형",브러쉬형,파우더형
0,28269721554,검은콩 딥 스케일링 나이트 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 천연샴푸, 비듬샴푸, 두피샴푸, 한방샴푸, 퍼퓸샴푸, 펌프형, 저자극, 촉...",True,True,...,False,False,False,False,False,False,False,False,False,False
1,28269744554,검은콩 보습 진정 모닝 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 모든두피용, 모든 모발용, 일반샴푸, 펌프형, 머릿결개선, 세정력, 풍부한...",True,False,...,False,False,False,False,False,False,False,False,False,False
2,29502248618,어글리솝 더 님 민감성용 97g,화장품/미용,헤어케어,샴푸,프롬더랜드,프롬더랜드,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급),...",True,False,...,False,False,False,False,False,False,False,False,False,False
3,20991784508,녹차실감 산뜻한 타입 지성 모발용 샴푸 500g,화장품/미용,헤어케어,샴푸,아모스,아모스,"[샴푸, 지성, 두피샴푸, 펌프형, 세정력, 상쾌함, 저자극, 촉촉함(수분공급), ...",True,False,...,False,False,False,False,False,False,False,False,False,False
4,29124648587,프로체인지 블랙샴푸 300g,화장품/미용,헤어케어,샴푸,더마밀,모다모다,"[샴푸, 탈모샴푸, 펌프형, 저자극, 계면활성제 프리, 파라벤 프리, 실리콘 프리,...",True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5635,21542720229,HR.101 딥케일링 샴푸 250ml,화장품/미용,헤어케어,샴푸,유씨엘,올리패스,"[샴푸, 일반샴푸, 세정력, 저자극, 촉촉함(수분공급), 각질케어, 실리콘 프리, ...",True,False,...,False,False,False,False,False,False,False,False,False,False
5636,35269230620,나드 샴푸 티트리로즈마리 900ml,화장품/미용,헤어케어,샴푸,서울화장품,부케가르니,"[샴푸, 일반샴푸, 펌프형, 머릿결개선, 세정력, 촉촉함(수분공급)]",True,False,...,False,False,False,False,False,False,False,False,False,False
5637,34833591621,아르간오일 쉐어 샴푸 1000ml,화장품/미용,헤어케어,샴푸,이볼루덤,이볼루덤,"[샴푸, 일반샴푸, 펌프형, 세정력, 풍부한 거품, 촉촉함(수분공급), 모근강화, ...",True,False,...,False,False,False,False,False,False,False,False,False,False
5638,36971151620,워터 리빌딩 샴푸 허브향 500ml,화장품/미용,헤어케어,샴푸,코스맥스,에이바이봄,"[샴푸, 일반샴푸, 머릿결개선, 촉촉함(수분공급), 모발영양]",True,False,...,False,False,False,False,False,False,False,False,False,False


In [62]:
def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:80]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/80)
    
    df = makePageDF(category_id,1)
    

    if total_page >1:
        for i in range(2,total_page+1):
            new_df = makePageDF(category_id,i)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [63]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [64]:
df

,ID,상품명,상품 카테고리 대분류,상품 카테고리 중분류,상품 카테고리 소분류,제조사,브랜드,특징,샴푸,천연샴푸,...,보색샴푸,쿨샴푸,주니어겸용,쉐도우형,무스형,스틱형,염색 모발용,"시트,티슈형",브러쉬형,파우더형
0,28269721554,검은콩 딥 스케일링 나이트 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 천연샴푸, 비듬샴푸, 두피샴푸, 한방샴푸, 퍼퓸샴푸, 펌프형, 저자극, 촉...",True,True,...,False,False,False,False,False,False,False,False,False,False
1,28269744554,검은콩 보습 진정 모닝 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 모든두피용, 모든 모발용, 일반샴푸, 펌프형, 머릿결개선, 세정력, 풍부한...",True,False,...,False,False,False,False,False,False,False,False,False,False
2,29502248618,어글리솝 더 님 민감성용 97g,화장품/미용,헤어케어,샴푸,프롬더랜드,프롬더랜드,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급),...",True,False,...,False,False,False,False,False,False,False,False,False,False
3,20991784508,녹차실감 산뜻한 타입 지성 모발용 샴푸 500g,화장품/미용,헤어케어,샴푸,아모스,아모스,"[샴푸, 지성, 두피샴푸, 펌프형, 세정력, 상쾌함, 저자극, 촉촉함(수분공급), ...",True,False,...,False,False,False,False,False,False,False,False,False,False
4,29124648587,프로체인지 블랙샴푸 300g,화장품/미용,헤어케어,샴푸,더마밀,모다모다,"[샴푸, 탈모샴푸, 펌프형, 저자극, 계면활성제 프리, 파라벤 프리, 실리콘 프리,...",True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5635,21542720229,HR.101 딥케일링 샴푸 250ml,화장품/미용,헤어케어,샴푸,유씨엘,올리패스,"[샴푸, 일반샴푸, 세정력, 저자극, 촉촉함(수분공급), 각질케어, 실리콘 프리, ...",True,False,...,False,False,False,False,False,False,False,False,False,False
5636,35269230620,나드 샴푸 티트리로즈마리 900ml,화장품/미용,헤어케어,샴푸,서울화장품,부케가르니,"[샴푸, 일반샴푸, 펌프형, 머릿결개선, 세정력, 촉촉함(수분공급)]",True,False,...,False,False,False,False,False,False,False,False,False,False
5637,34833591621,아르간오일 쉐어 샴푸 1000ml,화장품/미용,헤어케어,샴푸,이볼루덤,이볼루덤,"[샴푸, 일반샴푸, 펌프형, 세정력, 풍부한 거품, 촉촉함(수분공급), 모근강화, ...",True,False,...,False,False,False,False,False,False,False,False,False,False
5638,36971151620,워터 리빌딩 샴푸 허브향 500ml,화장품/미용,헤어케어,샴푸,코스맥스,에이바이봄,"[샴푸, 일반샴푸, 머릿결개선, 촉촉함(수분공급), 모발영양]",True,False,...,False,False,False,False,False,False,False,False,False,False


In [65]:
def makePageDF(category_id, page_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : '60',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:60]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '60',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    total_page = math.ceil(total/60)
    
    df = makePageDF(category_id,1)
    
    if total_page >1:
        for i in range(2,total_page+1):
            new_df = makePageDF(category_id,i)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [66]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [67]:
df

,ID,상품명,상품 카테고리 대분류,상품 카테고리 중분류,상품 카테고리 소분류,제조사,브랜드,특징,샴푸,천연샴푸,...,보색샴푸,쿨샴푸,주니어겸용,쉐도우형,무스형,스틱형,염색 모발용,"시트,티슈형",브러쉬형,파우더형
0,28269721554,검은콩 딥 스케일링 나이트 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 천연샴푸, 비듬샴푸, 두피샴푸, 한방샴푸, 퍼퓸샴푸, 펌프형, 저자극, 촉...",True,True,...,False,False,False,False,False,False,False,False,False,False
1,28269744554,검은콩 보습 진정 모닝 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 모든두피용, 모든 모발용, 일반샴푸, 펌프형, 머릿결개선, 세정력, 풍부한...",True,False,...,False,False,False,False,False,False,False,False,False,False
2,29502248618,어글리솝 더 님 민감성용 97g,화장품/미용,헤어케어,샴푸,프롬더랜드,프롬더랜드,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급),...",True,False,...,False,False,False,False,False,False,False,False,False,False
3,20991784508,녹차실감 산뜻한 타입 지성 모발용 샴푸 500g,화장품/미용,헤어케어,샴푸,아모스,아모스,"[샴푸, 지성, 두피샴푸, 펌프형, 세정력, 상쾌함, 저자극, 촉촉함(수분공급), ...",True,False,...,False,False,False,False,False,False,False,False,False,False
4,29124648587,프로체인지 블랙샴푸 300g,화장품/미용,헤어케어,샴푸,더마밀,모다모다,"[샴푸, 탈모샴푸, 펌프형, 저자극, 계면활성제 프리, 파라벤 프리, 실리콘 프리,...",True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5635,21542720229,HR.101 딥케일링 샴푸 250ml,화장품/미용,헤어케어,샴푸,유씨엘,올리패스,"[샴푸, 일반샴푸, 세정력, 저자극, 촉촉함(수분공급), 각질케어, 실리콘 프리, ...",True,False,...,False,False,False,False,False,False,False,False,False,False
5636,36971151620,워터 리빌딩 샴푸 허브향 500ml,화장품/미용,헤어케어,샴푸,코스맥스,에이바이봄,"[샴푸, 일반샴푸, 머릿결개선, 촉촉함(수분공급), 모발영양]",True,False,...,False,False,False,False,False,False,False,False,False,False
5637,34833591621,아르간오일 쉐어 샴푸 1000ml,화장품/미용,헤어케어,샴푸,이볼루덤,이볼루덤,"[샴푸, 일반샴푸, 펌프형, 세정력, 풍부한 거품, 촉촉함(수분공급), 모근강화, ...",True,False,...,False,False,False,False,False,False,False,False,False,False
5638,35269230620,나드 샴푸 티트리로즈마리 900ml,화장품/미용,헤어케어,샴푸,서울화장품,부케가르니,"[샴푸, 일반샴푸, 펌프형, 머릿결개선, 세정력, 촉촉함(수분공급)]",True,False,...,False,False,False,False,False,False,False,False,False,False


In [70]:
def makePageDF(category_id, page_id, pagingsize):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : str(pagingsize),
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:pagingsize]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    
    df = pd.DataFrame()
    
    for j in [20,40,60,80]:
        total_page = math.ceil(total/j)
        for i in range(1,total_page+1):
            new_df = makePageDF(category_id,i,j)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [71]:
df, merged_df, attributes = makeCategoryDF(100001011)

In [72]:
df

,ID,상품명,상품 카테고리 대분류,상품 카테고리 중분류,상품 카테고리 소분류,제조사,브랜드,특징,샴푸,천연샴푸,...,보색샴푸,쿨샴푸,주니어겸용,쉐도우형,무스형,스틱형,염색 모발용,"시트,티슈형",브러쉬형,파우더형
0,28269721554,검은콩 딥 스케일링 나이트 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 천연샴푸, 비듬샴푸, 두피샴푸, 한방샴푸, 퍼퓸샴푸, 펌프형, 저자극, 촉...",True,True,...,False,False,False,False,False,False,False,False,False,False
1,28269744554,검은콩 보습 진정 모닝 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 모든두피용, 모든 모발용, 일반샴푸, 펌프형, 머릿결개선, 세정력, 풍부한...",True,False,...,False,False,False,False,False,False,False,False,False,False
2,29502248618,어글리솝 더 님 민감성용 97g,화장품/미용,헤어케어,샴푸,프롬더랜드,프롬더랜드,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급),...",True,False,...,False,False,False,False,False,False,False,False,False,False
3,20991784508,녹차실감 산뜻한 타입 지성 모발용 샴푸 500g,화장품/미용,헤어케어,샴푸,아모스,아모스,"[샴푸, 지성, 두피샴푸, 펌프형, 세정력, 상쾌함, 저자극, 촉촉함(수분공급), ...",True,False,...,False,False,False,False,False,False,False,False,False,False
4,29124648587,프로체인지 블랙샴푸 300g,화장품/미용,헤어케어,샴푸,더마밀,모다모다,"[샴푸, 탈모샴푸, 펌프형, 저자극, 계면활성제 프리, 파라벤 프리, 실리콘 프리,...",True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16885,32237760627,니욕 샴푸바 비타미나 80g,화장품/미용,헤어케어,샴푸,,,"[샴푸, 샴푸바, 고체형, 머릿결개선, 세정력, 풍부한 거품, 상쾌함, 저자극, 촉...",True,False,...,False,False,False,False,False,False,False,False,False,False
21768,33587362619,밸런싱 샴푸 로즈 무드 1000ml,화장품/미용,헤어케어,샴푸,세화피앤씨,메리뮤,"[샴푸, 일반샴푸, 펌프형, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급), 약...",True,False,...,False,False,False,False,False,False,False,False,False,False
22009,12994368123,비오길프 핑크 바니타스 센서티브 샴푸 250ml,화장품/미용,헤어케어,샴푸,,,"[샴푸, 촉촉함(수분공급)]",True,False,...,False,False,False,False,False,False,False,False,False,False
22403,36670520620,컬러 여신 샴푸 염색모용 400ml,화장품/미용,헤어케어,샴푸,TIGI,TIGI,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 손상케어, 모발영양]",True,False,...,False,False,False,False,False,False,False,False,False,False


In [77]:
def makePageDF(category_id, page_id, pagingsize):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : str(page_id),
        'pagingSize' : str(pagingsize),
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    print(resp.request.url)
    
    # 각 상품의 정보를 저장할 리스트를 생성합니다.
    products_info = []

    for item in itemList['list'][:pagingsize]:

        attributes_dict = dict()
        # 'attributeValue'와 'characterValue' 문자열을 리스트로 분리합니다.
        attribute_values = item['item']['attributeValue'].split('|')
        character_values = item['item']['characterValue'].split('|')

        # 'attributeValue' 리스트의 각 요소에서 '_M' 문자열을 제거합니다.
        attribute_values = [value.replace('_M', '') for value in attribute_values]

        min_len = min(len(attribute_values), len(character_values))

        for j in range(min_len):
            attribute = attribute_values[j]
            character = character_values[j]

            if attribute in attributes_dict:
                attributes_dict[attribute].append(character)
            else:
                attributes_dict[attribute] = [character]

        attribute_list = [item['item']['category3Name']]

        for i in attributes_dict:
            if i not in ['용량', '구성', '']:
                attribute_list = attribute_list + attributes_dict[i]
            else:
                continue

        dict_data = {
            'ID': item['item']['id'],
            '상품명': item['item']['productName'],
            '상품 카테고리 대분류': item['item']['category1Name'],
            '상품 카테고리 중분류': item['item']['category2Name'],
            '상품 카테고리 소분류': item['item']['category3Name'],
            '제조사': item['item']['maker'],
            '브랜드': item['item']['brand'],
            '특징': attribute_list
        }
        for attribute in attribute_list:
            dict_data[attribute] = True

        products_info.append(dict_data)
        
        # 딕셔너리의 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(products_info)
    
    return df

def makeCategoryDF(category_id):
    url = 'https://search.shopping.naver.com/search/category/' + str(category_id)
    params = {
        'pagingIndex' : '1',
        'pagingSize' : '80',
        'productSet' : 'model'
    }
    headers = {'User-Agent':generate_user_agent(os =('mac', 'linux'),device_type='desktop')}
    resp = request('GET',url=url, params=params, headers=headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    script_tag = dom.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.text)
    itemList = data['props']['pageProps']['initialState']['products']
    total = data['props']['pageProps']['initialState']['products']['total']
    
    df = pd.DataFrame()
    
    for j in [60,80]:
        total_page = math.ceil(total/j)
        for i in range(1,total_page+1):
            new_df = makePageDF(category_id,i,j)
            df = pd.concat([df, new_df], ignore_index=True)
    
    df = df.drop_duplicates(subset=['ID'])
    df = pd.DataFrame(df).astype({'제조사': 'category'})
    df = pd.DataFrame(df).astype({'브랜드': 'category'})
    attribute_columns = df.columns.drop(['ID', '상품명', '상품 카테고리 대분류', '상품 카테고리 중분류','상품 카테고리 소분류','제조사','브랜드','특징'])
    df[attribute_columns] = df[attribute_columns].fillna(False).astype('bool')
    small_df = df.explode('특징')[['ID','상품명','특징']]
    attributes = pd.DataFrame(small_df['특징'].unique()).reset_index()
    attributes.columns = ['attribute_id', '특징']
    attributes = pd.DataFrame(attributes).astype({'attribute_id': 'category'})
    merged_df = pd.merge(small_df, attributes, on=['특징'])
    
    return df, merged_df, attributes

In [78]:
df, merged_df, attributes = makeCategoryDF(100001011)

https://search.shopping.naver.com/search/category/100001011?pagingIndex=1&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=2&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=3&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=4&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=5&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=6&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=7&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=8&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=9&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/10000

https://search.shopping.naver.com/search/category/100001011?pagingIndex=79&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=80&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=81&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=82&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=83&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=84&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=85&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=86&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=87&pagingSize=60&productSet=model
https://search.shopping.naver.com/search/categ

https://search.shopping.naver.com/search/category/100001011?pagingIndex=62&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=63&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=64&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=65&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=66&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=67&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=68&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=69&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/category/100001011?pagingIndex=70&pagingSize=80&productSet=model
https://search.shopping.naver.com/search/categ

In [79]:
df

,ID,상품명,상품 카테고리 대분류,상품 카테고리 중분류,상품 카테고리 소분류,제조사,브랜드,특징,샴푸,천연샴푸,...,보색샴푸,쿨샴푸,주니어겸용,쉐도우형,무스형,스틱형,염색 모발용,"시트,티슈형",브러쉬형,파우더형
0,28269721554,검은콩 딥 스케일링 나이트 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 천연샴푸, 비듬샴푸, 두피샴푸, 한방샴푸, 퍼퓸샴푸, 펌프형, 저자극, 촉...",True,True,...,False,False,False,False,False,False,False,False,False,False
1,28269744554,검은콩 보습 진정 모닝 샴푸 로즈마리향 400ml,화장품/미용,헤어케어,샴푸,동방코스메틱,올솔,"[샴푸, 모든두피용, 모든 모발용, 일반샴푸, 펌프형, 머릿결개선, 세정력, 풍부한...",True,False,...,False,False,False,False,False,False,False,False,False,False
2,29502248618,어글리솝 더 님 민감성용 97g,화장품/미용,헤어케어,샴푸,프롬더랜드,프롬더랜드,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급),...",True,False,...,False,False,False,False,False,False,False,False,False,False
3,20991784508,녹차실감 산뜻한 타입 지성 모발용 샴푸 500g,화장품/미용,헤어케어,샴푸,아모스,아모스,"[샴푸, 지성, 두피샴푸, 펌프형, 세정력, 상쾌함, 저자극, 촉촉함(수분공급), ...",True,False,...,False,False,False,False,False,False,False,False,False,False
4,13023412774,EM 발효 다시마 천연샴푸 500ml,화장품/미용,헤어케어,샴푸,이엔에스코리아,청미정,"[샴푸, 모든두피용, 모든 모발용, 천연샴푸, 두피샴푸, 펌프형, 세정력, 풍부한 ...",True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11217,36469650621,샴푸 다크로즈 520ml,화장품/미용,헤어케어,샴푸,뉴겐코스메틱,말콤,"[샴푸, 일반샴푸, 세정력, 풍부한 거품, 모발영양]",True,False,...,False,False,False,False,False,False,False,False,False,False
11225,36088691621,아몬드 오일 블렌드 샴푸 354ml,화장품/미용,헤어케어,샴푸,,아비노,"[샴푸, 크림형, 머릿결개선, 세정력, 풍부한 거품, 상쾌함, 저자극, 촉촉함(수분...",True,False,...,False,False,False,False,False,False,False,False,False,False
11239,36671634618,컬러 브릴리언스 샴푸 300ml,화장품/미용,헤어케어,샴푸,컨,컨,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 촉촉함(수분공급),...",True,False,...,False,False,False,False,False,False,False,False,False,False
11256,34025742618,크리에벌 물비누 500ml,화장품/미용,헤어케어,샴푸,,서원알로에,"[샴푸, 일반샴푸, 머릿결개선, 세정력, 풍부한 거품, 저자극, 모발영양]",True,False,...,False,False,False,False,False,False,False,False,False,False
